# North SIE (daily)

In [61]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np

## MODEL 1: Linear model by year averaged
### (Baseline Model)

- Need a consistent test set to measure performance throughout model comparison- perhaps training on sea-ice data up to year 2011, and evaluating on years 2011-2025

- Metrics I want to explicitly test are rmse and mae

- Need to consider how my test set selection impacts extrapolation when investigating first ice-free year

In [62]:
df = pd.read_csv("data/N_seaice_extent_daily_v4.0.csv", skiprows=1) #skipping 1st row

df.columns = df.columns.str.strip()
df = df.rename(columns={
    "YYYY": "Year",
    "MM": "Month",
    "DD": "Day",
    "10^6 sq km": "Extent",
    "10^6 sq km.1": "Missing"
})

df = df.drop(columns=[col for col in df.columns if "Source data" in col]) #dropping source data column
yearly_avg = df.groupby('Year')['Extent'].mean().reset_index()
print(yearly_avg.head())

#splitting into train and test sets (training up to 2011)
train = yearly_avg[yearly_avg["Year"] <= 2011]
X_train = train["Year"]. values.reshape(-1,1)
y_train = train["Extent"].values

test = yearly_avg[yearly_avg["Year"] > 2011]
X_test = test["Year"]. values.reshape(-1,1)
y_test = test["Extent"].values

   Year     Extent
0  1978  12.487000
1  1979  12.319560
2  1980  12.334148
3  1981  12.135486
4  1982  12.439445


In [63]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"mae = {mae}, rmse={rmse}, r2= {r2}")

results = {
    "model": "Linear Regression",
    "features": "Year",
    "target": "Yearly averaged SIE",
    "train_period": "1978-2011",
    "test_period": "2012-2025",
    "MAE": mae,
    "RMSE": rmse,
    "R2": r2
}


#adding results dictionary to a new data frame so that i can easily compare models throughout
results_df = pd.DataFrame([results])

#converting it into a csv file
results_df.to_csv(
    "regression_model_results.csv",
    mode = "a",
    header=not pd.io.common.file_exists("regression_model_results.csv"),
    index=False
)


mae = 0.2083645120170348, rmse=0.2419211873045379, r2= -0.02499090472636145


## MODEL 2: Linear model by monthly average
### (2nd Baseline Model)

- Target is now monthly averaged rather than yearly

In [65]:
df.head()

monthly_average = df.groupby(['Year', 'Month'])['Extent'].mean().reset_index()
monthly_average.head()


train2 = monthly_average[monthly_average["Year"] <= 2011]
X_train2 = train2.drop(columns=["Extent"]).values
y_train2 = train2["Extent"].values

test2 = monthly_average[monthly_average["Year"] > 2011]
X_test2 = test2.drop(columns=["Extent"]).values
y_test2 = test2["Extent"].values

In [66]:
model = LinearRegression()
model.fit(X_train2, y_train2)
y_pred2 = model.predict(X_test2)

mae = mean_absolute_error(y_test2, y_pred2)
rmse = np.sqrt(mean_squared_error(y_test2, y_pred2))
r2 = r2_score(y_test2, y_pred2)

print(f"mae = {mae}, rmse={rmse}, r2= {r2}")

results = {
    "model": "Linear Regression",
    "features": "Year, Month",
    "target": "Monthly averaged SIE for each year",
    "train_period": "1978-2011",
    "test_period": "2012-2025",
    "MAE": mae,
    "RMSE": rmse,
    "R2": r2
}


#adding results dictionary to a new data frame so that i can easily compare models throughout
results_df = pd.DataFrame([results])

#converting it into a csv file
results_df.to_csv(
    "regression_model_results.csv",
    mode = "a",
    header=not pd.io.common.file_exists("regression_model_results.csv"),
    index=False
)

mae = 2.1808971714695016, rmse=2.610417471637942, r2= 0.4338773239433915


- Larger average error but higher r2 score
- Larger variability for monthly-averaged data --> larger average errors
- However relative to this variability, the model accounts for a fair proportion of it 

## Generating Lag Features

- Generating features to keep track of previous 3 months, half-year, 1 year and 2 years ago
- Referred to as Lag 1, Lag 2, Lag 3, Lag 6, Lag 12, Lag 24

In [69]:
monthly_average.head()
df_lagged = monthly_average.copy()

lags = [1,2,3,6,12,24]

for lag in lags:
    df_lagged[f'Lag {lag}'] = df_lagged['Extent'].shift(lag)


df_lagged = df_lagged.dropna()
df_lagged.head()

,Year,Month,Extent,Lag 1,Lag 2,Lag 3,Lag 6,Lag 12,Lag 24
24,1980,10,9.182750,7.667067,7.984267,10.100062,15.429067,8.747937,10.402667
25,1980,11,11.382867,9.182750,7.667067,7.984267,13.792600,10.943067,11.645133
26,1980,12,13.592933,11.382867,9.182750,7.667067,12.204600,13.336267,13.667063
27,1981,1,14.909688,13.592933,11.382867,9.182750,10.100062,14.861875,15.414000
28,1981,2,15.604071,14.909688,13.592933,11.382867,7.984267,15.955143,16.175286


In [70]:
train3 = df_lagged[df_lagged["Year"] <= 2011]
X_train3 = train3.drop(columns=["Extent"]).values
y_train3 = train3["Extent"].values

test3 = df_lagged[df_lagged["Year"] > 2011]
X_test3 = test3.drop(columns=["Extent"]).values
y_test3 = test3["Extent"].values

print("Train years:", train3["Year"].unique())
print("Test years:", test3["Year"].unique())

Train years: [1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993
 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007
 2008 2009 2010 2011]
Test years: [2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024 2025]


In [71]:
model = LinearRegression()
model.fit(X_train3, y_train3)
y_pred3 = model.predict(X_test3)

mae = mean_absolute_error(y_test3, y_pred3)
rmse = np.sqrt(mean_squared_error(y_test3, y_pred3))
r2 = r2_score(y_test3, y_pred3)

print(f"mae = {mae}, rmse={rmse}, r2= {r2}")

results = {
    "model": "Linear Regression",
    "features": "Year, Month, Lags",
    "target": "Monthly averaged SIE for each year",
    "train_period": "1980-2011",
    "test_period": "2012-2025",
    "MAE": mae,
    "RMSE": rmse,
    "R2": r2
}


#adding results dictionary to a new data frame so that i can easily compare models throughout
results_df = pd.DataFrame([results])

#converting it into a csv file
results_df.to_csv(
    "regression_model_results.csv",
    mode = "a",
    header=not pd.io.common.file_exists("regression_model_results.csv"),
    index=False
)

mae = 0.2840172055801048, rmse=0.38597079969389864, r2= 0.9876234524341628


- Much larger r2 score and lower average errors compared to 2nd baseline model
- Worth noting that we had less training data as a result of the lag generation

## Introducing 12 month rolling average and time index features

- Taking into account ...

In [74]:
df_lagged2 = df_lagged.copy()

df_lagged2['Rolling Yearly Mean'] = df_lagged2['Extent'].shift(1).rolling(12).mean()
df_lagged2['Time Index'] = range(len(df_lagged2))

df_lagged2 = df_lagged2.dropna()
df_lagged2.head()

,Year,Month,Extent,Lag 1,Lag 2,Lag 3,Lag 6,Lag 12,Lag 24,Rolling Yearly Mean,Time Index
36,1981,10,8.856267,7.138467,7.844313,10.271000,15.009933,9.182750,8.747937,12.233243,12
37,1981,11,10.929000,8.856267,7.138467,7.844313,13.801625,11.382867,10.943067,12.206036,13
38,1981,12,13.341125,10.929000,8.856267,7.138467,12.429733,13.592933,13.336267,12.168214,14
39,1982,1,15.176733,13.341125,10.929000,8.856267,10.271000,14.909688,14.861875,12.147230,15
40,1982,2,15.973500,15.176733,13.341125,10.929000,7.844313,15.604071,15.955143,12.169483,16


In [75]:
train4 = df_lagged2[df_lagged2["Year"] <= 2011]
X_train4 = train4.drop(columns=["Extent"]).values
y_train4 = train4["Extent"].values

test4 = df_lagged2[df_lagged2["Year"] > 2011]
X_test4 = test4.drop(columns=["Extent"]).values
y_test4 = test4["Extent"].values

print("Train years:", train4["Year"].unique())
print("Test years:", test4["Year"].unique())

Train years: [1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994
 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008
 2009 2010 2011]
Test years: [2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024 2025]


In [76]:
model = LinearRegression()
model.fit(X_train4, y_train4)
y_pred4 = model.predict(X_test4)

mae = mean_absolute_error(y_test4, y_pred4)
rmse = np.sqrt(mean_squared_error(y_test4, y_pred4))
r2 = r2_score(y_test4, y_pred4)

print(f"mae = {mae}, rmse={rmse}, r2= {r2}")

results = {
    "model": "Linear Regression",
    "features": "Year, Month, Lags, Rolling Yearly Average, Time Index",
    "target": "Monthly averaged SIE for each year",
    "train_period": "1981-2011",
    "test_period": "2012-2025",
    "MAE": mae,
    "RMSE": rmse,
    "R2": r2
}


#adding results dictionary to a new data frame so that i can easily compare models throughout
results_df = pd.DataFrame([results])

#converting it into a csv file
results_df.to_csv(
    "regression_model_results.csv",
    mode = "a",
    header=not pd.io.common.file_exists("regression_model_results.csv"),
    index=False
)

mae = 0.28715445058368044, rmse=0.3865895159290696, r2= 0.9875837410958257


- Very similar results to previous